In [ ]:
""" Abstract

The demo is a prototype of the project model. Codes here could change in the future.
The main point of codes below is to run on local computer and test whether it works on small scale of data.

"""

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from elmoformanylangs import Embedder

In [ ]:
""" Simple Seqence to Sequence Implementation

A simple implementation of Sequence to Sequence model. It works as baseline

Args:
    input_dim:  dimension of input word vector.
    output_dim: dimension of output word vector.
    hidden_dim: dimension of hidden states vector.

Returns:
    model: the whole model of simple Seq2Seq model.

"""
def simpleSeq2Seq( output_dim = 1024, input_dim = 1024, hidden_dim = 512, name = "" ):
    # Encoder
    encoder_input = Input( shape = ( None, input_dim ), name = "encoder_input" )
    encoder_output, state_h, state_c = LSTM( hidden_dim, return_state = True )( ori_input )
    state_encoder = [state_h, state_c]
    # Decoder
    decoder_input = Input( shape = ( None, output_dim ), name = "decoder_input" )
    decoder_outputs = LSTM( hidden_dim, return_sequences = True )( decoder_input, initial_state = state_encoder )
    decoder_outputs = Dense( output_dim, activation = "softmax", name = "decoder_output" )( decoder_outputs )
    model = Model( Input[encoder_input, decoder_input], output = decoder_outputs, name = name )
    return model

def model( langA_dim = 1024, langB_dim = 1024, hidden_dim = 512, lmA, lmB ):
    # first part of translation model from language A to language B
    langA_input  = Input( shape = ( None, langA_dim ), name = "langA_input"  )
    langB_output = Input( shape = ( None, langB_dim ), name = "langB_output" )
    A2B_model = simpleSeq2Seq( langB_dim, langA_dim, hidden_dim, name = "A2B" )
    langB_output = A2B_model( [langA_input, langB_output] )
    # language model <- langB_output
    # second part of translation model from another language to original language
    langB_input  = Input( shape = ( None, langB_dim ), name = "langB_input"  )
    langA_output = Input( shape = ( None, langA_dim ), name = "langA_output" )
    B2A_model = simpleSeq2Seq( langA_dim, langB_dim, hidden_dim, name = "B2A" )
    langA_output = B2A_model( [langB_input, langA_output] )
    # language model <- langA_output